In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
# Q1. What's the standard deviation of the predicted duration for this dataset?

print(f"Standard deviation of predictions: {round(y_pred.std(axis=0), 2)}")

Standard deviation of predictions: 6.25


In [9]:
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame(data={"ride_id": df['ride_id'], "predictions": y_pred})
output_file = "predictions.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [11]:
# Q2. What's the size of the output file

!ls -sh $output_file

66M predictions.parquet


In [ ]:
# Q3. Creating the scoring script
# jupyter nbconvert --to script starter.ipynb

In [5]:
# Q4
!pipenv install scikit-learn==1.5.0 pyarrow pandas --python=3.10

# What's the first hash for the Scikit-Learn dependency? "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"

Installing scikit-learn==1.5.0...
Resolving scikit-learn==1.5.0...
✔ Installation Succeeded
⠋ Installing scikit-learn...
Installing pyarrow...
Resolving pyarrow...
✔ Installation Succeeded
⠋ Installing pyarrow...
Installing pandas...
Resolving pandas...
Added pandas to Pipfile's [packages] ...
✔ Installation Succeeded...
⠋ Installing pandas...
Pipfile.lock (c38461) out of date, updating to (3c2ffe)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠧ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (2781451e8c683e398151f1d8c3d80084214c5cb95e0d9a8227dc69bc2f3c2ffe)!
Installing dependencies from Pipfile.lock (3c2ffe)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [6]:
# Q5 Parametrize the script
!pipenv run python starter.py yellow 2023 4

loading model
reading file: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet
applying model
saving predictions
Mean of predictions: 14.29
